In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize ,
poly)
from sklearn.model_selection import train_test_split

In [2]:
from functools import partial
from sklearn.model_selection import \
(cross_validate ,
KFold ,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

### Validation Set Approach

In [3]:
### We want to estimate the test error rates that results from fitting various linear models on the auto data set
Auto = load_data('Auto')
Auto_train , Auto_valid = train_test_split(Auto , test_size=196, random_state=0)

In [4]:
# we fit the linear regression using just the training dataset
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train , X_train)
results = model.fit()

In [5]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
# MSE of the model
np.mean((y_valid - valid_pred)**2)

23.61661706966988

In [6]:
### We can also estimate the validation error for higher degree polynomial regressions by creating the following function
def evalMSE(terms ,response ,train , test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train , X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [7]:
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                        'mpg',
                        Auto_train ,
                        Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [8]:
Auto_train , Auto_valid = train_test_split(Auto ,test_size=196, random_state=3)
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],'mpg',Auto_train , Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

In [9]:
# sklearn_sm is a wrapper that enables us to easily use cross_validation tools.
# its first argument is a model from the library statsmodels; 
# it can take two additional arguments: model_str (to specify a formula) or model_args (dictionary of additional arguments to specify a family argument)

hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']

# the arguments of cross_validate are: object with proper fit, predict or score methods; X features; Y response;
#cv specifies K results in K-fold cross validation
cv_results = cross_validate(hp_model, X, Y, cv = Auto.shape[0])

cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792923

In [10]:
# We automate the procedure for increasingly complex polynomial fits
#We use the outer method after a math operation; 
# it takes two arrays as arguments and then forms a larger array where the operationis applied to each pair of elements of the two arrays.

cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d + 1))
    M_CV = cross_validate(M, X, Y, cv = Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443034, 19.0332226 ])

In [11]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In [12]:
# Now we use KFold to partition with cv < n

cv_error = np.zeros(5)
cv = KFold(n_splits=10, shuffle=True, random_state=0)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv = cv)
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848404, 19.13720771])

In [14]:
# ShuffleSplit function helps in implementing the validation set approach as K-Fold cross-validation
validation = ShuffleSplit(n_splits = 1, test_size = 196, random_state = 0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis = 1), Auto['mpg'], cv=validation);
results['test_score']

array([23.61661707])

In [15]:
# we estimate the variability in the test error
validation = ShuffleSplit(n_splits = 10, test_size = 196, random_state = 0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis = 1), Auto['mpg'], cv = validation)

# the standard deviation is not a valid estimate of the sampling variability of the mean test score or the individual scores;
# this is due to the randomly-selected training samples which overlaps hence introduces correlation;
# however, it gives an idea of the MonteCarlo variation
results['test_score'].mean(), results['test_score'].std()

(23.802232661034168, 1.421845094109181)

#### Bootstrap

In [18]:
# We define a function for its use in estimating standard error when our data is stored in a dataframe
# The goal is to estimate the sampling variance of the parameter alpha;
# We assume the dataframe D to have column X and Y, as well as a vector idxindicating which observations should be used to estimate alpha.
# The function returns an estimate for alpha based on the formula seen in theory
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X', 'Y']].loc[idx], rowvar = False)
    return ((cov_[1,1] - cov_[0,1]) / (cov_[0,0] + cov_[1,1] - 2*cov_[0,1]))

In [17]:
# I apply the function to the loaded Dataframe considering all 100 observations
alpha_func(Portfolio, range(100))

0.57583207459283

In [20]:
# We now randomly select 100 observations from range(100), but with replacement as true, so that a new dataset is built
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace = True))

0.6074452469619002

In [21]:
# We can generalize the process to create a simple function boot_SE()
def boot_SE(func, D, n=None, B=1000, seed = 0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B) **2)

In [22]:
# we try the function to evaluate the accuracy of our estimate of alpha using B = 1000;
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed = 0)
alpha_SE

0.09118176521277516

In [26]:
### Now we use the bootrastrap approach to estimate the variability of the coefficient estimates and predictions from a statistical learning method;
# we use clone function to make a copy of the formula that can be refit to the new dataframe
# in this case we created a specific bootstraping for regression models
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [28]:
# the first two arguments do not change --> we'd like to freeze them --> we use partial function from fucntools library;
# we do that to have ideal input for boot_SE func defined before;
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')
hp_func?

Signature:      hp_func(D, idx)
Call signature: hp_func(*args, **kwargs)
Type:           partial
String form:    functools.partial(<function boot_OLS at 0x0000016565206CA0>, ModelSpec(terms=['horsepower']), 'mpg')
File:           c:\users\gabri\appdata\local\programs\python\python311\lib\functools.py
Docstring:     
partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.

In [31]:
# now hp_func is ready to create bootsrap estimates;
# we test it on 10 samples
rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(392, 392, replace = True)) for _ in range(10)])

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [32]:
# now we use boot_SE to compute 1000 bootstrap estimates
hp_se = boot_SE(hp_func, Auto, B = 1000, seed = 10)
hp_se

intercept     0.848807
horsepower    0.007352
dtype: float64

In [36]:
#we compare results with the standard formulas;
# there's a difference, but bootstrap is more reliable, since it has less assumptions
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [37]:
# now we compute the bootstrap standard error estimates and the standard linear regression estimates that result from fitting the quadratic model

quad_model = MS([poly('horsepower', 2, raw = True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

In [38]:
#We compare results to the standard ones
M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64